In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle_data
import census_api
import requests
import json

In [12]:
def fips_code(fips_df,state):
    return fips_df.Code[(fips_df.Name==state) | (fips_df.Abbr==state)].iat[0]

In [13]:
fips_df = pickle_data.get_pickled_data('./fips.pickle')
print(fips_code(fips_df,'Georgia'), '==', fips_code(fips_df,'GA'))

13.0 == 13.0


In [89]:
def get_request(yr, ds, q_vars, preds=[], reg='*'):
    req = f'https://api.census.gov/data/{yr}/{ds}?get='
    req += ','.join(q_vars)
    if len(preds) > 0:
        req += '&'+'&'.join(preds)+'&'
    else:
        req += '&'
    req += f'for=state:{reg}&key={census_api.api_key}'
    return req

In [91]:
req = get_request(2014,'pep/natstprc',
                  ['STNAME','POP','DEATHS'],
                  ['DATE=7'])
sauce = requests.get(req)
pop_df = pd.read_json(json.dumps(sauce.json()))
pop_df.columns = pop_df.iloc[0].values
pop_df = pop_df[1:]
pop_df.state = pop_df.state.astype('float64')

In [92]:
pop_df = pop_df[pop_df.state.isin(fips_df.Code)]

In [93]:
from census import Census
from us import states

In [115]:
c = Census(census_api.api_key, year=2013)
c_df = pd.read_json(json.dumps(c.acs1.state(('NAME',
                                             'B11004_001E',
                                             'B11004_002E',
                                             'B11004_003E'),
                                            Census.ALL)))

In [133]:
req = get_request(2013,'acs1',
                  ['NAME',
                   'B11004_001E',
                   'B11004_002E',
                   'B11004_003E'])
sauce = requests.get(req)
r_df = pd.read_json(json.dumps(sauce.json()))
r_df.columns = r_df.iloc[0].values
r_df = r_df[1:]
for c in r_df.columns[~r_df.columns.isin(['NAME'])]:
    r_df[c] = r_df[c].astype('int64')